In [ ]:
# SymPy.det(convert(Array{Float64}, MPlus))
# MTest = deepcopy(MPlus)
# SymPy.det(MTest)
[MPlus[1] MPlus[2]; MPlus[3] MPlus[4]]
MPlusCopy = Array{SymPy.Sym}(2,2)
for i in 1:2
    for j in 1:2
        MPlusCopy[i,j] = MPlus[i,j]
    end
end
MPlusCopy
SymPy.det(MPlusCopy)
typeof([MPlus[1] MPlus[2]; MPlus[3] MPlus[4]])
# expand(MPlus[4]*MPlus[1] - MPlus[3]*MPlus[2])
expand(SymPy.det([MPlus[1] MPlus[2]; MPlus[3] MPlus[4]]))

In [ ]:
# function to clean up floating point numbers in (constants and coefficients of) the symbolic expression of an exponential polynomial
# helper function that deals with the case where func(expr) = func(sympyExpExpr)
function clean_up_float_exp(expr::SymPy.Sym)
    term = args(expr)[1]
    termCleaned = clean_up_float_power_add_mult(term)
    result = subs(expr,args(expr)[1],termCleaned)
    return result
end
# helper function that deals with cases where func(expr) = func(sympyPowerExpr), func(sympyAddExpr), func(sympyMultExpr)
function clean_up_float_power_add_mult(expr::SymPy.Sym)
    expr = expand(expr)
    terms = args(expr)
    # println("terms = $terms")
    result = 0
    # if the expanded expression contains toplevel power calculations, the expression must be of the form x^a, so terms = args(expr) = (x, a)
    if func(expr) == func(sympyPowerExpr)
        println("power!")
        power = terms[2]
        # powerCleaned = round(power, signif)
        powerCleaned = power
        result = terms[1]^powerCleaned

    # if the expanded expression contains toplevel addition, the individual terms must all be products
    elseif func(expr) == func(sympyAddExpr)
        println("addition!")
        result = 0
        for term in terms
            # if term is exponential
            if func(term) == func(sympyExpExpr)
                termCleaned = clean_up_float_exp(term)
                println("termCleaned = $termCleaned")
            # else, further split each product term into indivdual factors
            else
                (constant, factors) = factor_list(term)
                constantCleaned = round(constant, signif)
                termCleaned = constantCleaned
                for (factor, power) in factors
                    # powerCleaned = round(power, signif) # floating point powers may cause cleaned numbers to become dirty again for imaginary numbers, e.g., 0.124*I^(1.0) evaluates to 0.12399847344634*I
                    powerCleaned = power # assume integer powers, which should be true for exponential polynomials
                    factorCleaned = factor^powerCleaned
                    termCleaned = termCleaned * factorCleaned
                    # println("termCleaned = $termCleaned")
                end
            end
            result = result + termCleaned
        end
    # if the expanded expression contains toplevel multiplication, the individual terms must all be powers
    elseif func(expr) == func(sympyMultExpr)
        println("multiplication!")
        result = 1
        for term in terms
            # if term is exponential
            if func(term) == func(sympyExpExpr)
                termCleaned = clean_up_float_exp(term)
                println("termCleaned = $termCleaned")
            # else, further split each product term into indivdual factors
            else
               (constant, factors) = factor_list(term)
                constantCleaned = round(constant, signif)
                termCleaned = constantCleaned
                for (factor, power) in factors
                    # powerCleaned = round(power, signif)
                    powerCleaned = power
                    factorCleaned = factor^powerCleaned
                    termCleaned = termCleaned * factorCleaned
                    # println("termCleaned = $termCleaned")
                end 
            end
            result = result * termCleaned
        end
    end
    return result
end

function clean_up_float(expr::SymPy.Sym)
    expr = expand(expr)
    # if the expansion contains toplevel power calculations, addition, or multiplications, use the helper function directly
    if func(expr) in [func(sympyPowerExpr), func(sympyAddExpr), func(sympyMultExpr)]
        result = clean_up_float_power_add_mult(expr)
    # if the expanded expression is neither of the above, it must be a single term, e.g., x or cos(2x+1), which is a function wrapping around an expression; in this case, use the helper function to clean up the expression and feed it back to the function
    else
        term = args(expr)[1]
        termCleaned = clean_up_float_power_add_mult(term)
        result = subs(expr,args(expr)[1],termCleaned)
    end
    return result
end